In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("..")
from generateTrajectory import *
from armControl import *
from forwardKinematics import robot_config
from utils.motor_setup import Motors
import signal

import generateTrajectory
import matplotlib.pyplot as plt


In [2]:
def signal_handler(signal, frame):
    motors.tcp_close()
signal.signal(signal.SIGINT, signal_handler)
#script_dir = os.path.dirname(__file__)
#results_dir = os.path.join(script_dir, 'Current_data/')
#if not os.path.isdir(results_dir):
#os.makedirs(results_dir)
#Constants
socket_ip = '192.168.1.20'
socket_port = 1122
P = 0
PL = 0
I = 0
IL = 0
D = 0
motors = Motors(P ,PL ,I, IL ,D)

In [3]:
trajPlanner = trajectoryGenerator()

In [4]:
motors.tcp_init(socket_ip, socket_port)

Opening socket at ip: 192.168.1.20 using port: 1122


In [5]:
print("Arming motors now...")
motors.arm_motors()

Arming motors now...
initializing motors to [0 0 0 0 0 0 0 0]
Arming motors


In [9]:
dt = 0.01
runtime = 60
num_elements = int(runtime/dt)
setpoints = np.zeros((num_elements, 8))
for i in range(num_elements):
    setpoints[i,:] = (np.ones(8) * np.sin(i*dt*2*pi)*4000).astype(int)
    
start_time = time.time()
for i in range(num_elements):
    current_time = time.time()
    #print(enc_position)
    setpoint = setpoints[i]
    motors.command_motors(setpoint)
    time.sleep(dt)
    
trajectory = trajPlanner.createTrajectoryNumPoints(setpoint, np.zeros(8), int(1/dt)).astype(int)
for i in range(trajectory.shape[1]):
    setpoint = list(trajectory[:,i].astype(int))
    motors.command_motors(list(trajectory[:,i].astype(int)))
    time.sleep(dt)
    #print(setpoint)
